In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tf2onnx
import os

In [29]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
DATA_DIR = "skin_cancer" 
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

In [30]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    shuffle=False, 
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

Found 2967 files belonging to 3 classes.
Found 330 files belonging to 3 classes.


In [31]:
class_names = train_ds.class_names
print(f"Classes found: {class_names}")

Classes found: ['Benign keratosis-like lesions', 'Melanocytic nevi', 'Melanoma']


In [32]:
def build_transfer_model(num_classes=3):
    inputs = Input(shape=(224, 224, 3))

    
    x = layers.RandomFlip("horizontal")(inputs)
    x = layers.RandomRotation(0.1)(x)

    x = preprocess_input(x)

    base_model = ResNet50(include_top=False, weights='imagenet')
    base_model.trainable = False # Freeze the pretrained weights
    
    x = base_model(x, training=False)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)  
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

model = build_transfer_model(num_classes=len(class_names))

In [33]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2 )

Epoch 1/2
93/93 ━━━━━━━━━━━━━━━━━━━━ 26s 222ms/step - accuracy: 0.5829 - loss: 0.9678 - val_accuracy: 0.7152 - val_loss: 0.6160
Epoch 2/2
93/93 ━━━━━━━━━━━━━━━━━━━━ 17s 184ms/step - accuracy: 0.7933 - loss: 0.4765 - val_accuracy: 0.7515 - val_loss: 0.5899


In [34]:
# Define the input signature for the model
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input_image"),)
onnx_filename = "skin_cancer_resnet.onnx"

In [35]:
# Convert the model to ONNX format

model_proto, _ = tf2onnx.convert.from_keras(
    model, 
    input_signature=spec, 
    opset=13,
    output_path=onnx_filename
)

I0000 00:00:1764387271.104850 60973926 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1764387271.105486 60973926 single_machine.cc:374] Starting new session
I0000 00:00:1764387271.106414 60973926 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1764387271.106430 60973926 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
I0000 00:00:1764387273.627576 60973926 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1764387273.627626 60973926 pluggable_device_factory.cc:271] Created TensorFlow device (